# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [3]:
# import libraries
import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sqlalchemy import create_engine
from nltk.stem.porter import PorterStemmer

from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ruoru\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ruoru\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\ruoru\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [9]:
# load data from database
DATABASE_FILENAME = '../db.sqlite3'
TABLE_NAME = 'disaster_message'
engine = create_engine('sqlite:///' + DATABASE_FILENAME)
df = pd.read_sql_table(TABLE_NAME, engine)

In [7]:
X = df['message']
Y = df.iloc[:, 4:]
category_names = list(df.columns[4:])

In [36]:
X

0        Weather update - a cold front from Cuba that c...
1                  Is the Hurricane over or is it not over
2                          Looking for someone but no name
3        UN reports Leogane 80-90 destroyed. Only Hospi...
4        says: west side of Haiti, rest of the country ...
                               ...                        
26211    The training demonstrated how to enhance micro...
26212    A suitable candidate has been selected and OCH...
26213    Proshika, operating in Cox's Bazar municipalit...
26214    Some 2,000 women protesting against the conduc...
26215    A radical shift in thinking came about as a re...
Name: message, Length: 26216, dtype: object

In [12]:
df.columns[4:]

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

### 2. Write a tokenization function to process your text data

In [40]:
def tokenize(text):
    # normalize text and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
     # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()

    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [42]:
from sklearn.multioutput import MultiOutputClassifier

In [56]:
pipeline = Pipeline([
    ('vect',CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf',MultiOutputClassifier(RandomForestClassifier(n_jobs=-1))),
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [57]:
X_train,X_test,y_train,y_test = train_test_split(X,Y)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [58]:
%%time
pipeline.fit(X_train,y_train)

Wall time: 1min 45s


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x00000208A82C4AF0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(n_jobs=-1)))])

In [59]:
%%time
y_pred = pipeline.predict(X_test)

Wall time: 4.66 s


In [64]:
from sklearn.metrics import classification_report,accuracy_score

In [61]:
y_pred.shape,y_test.shape,len(list(Y.columns))

((6554, 36), (6554, 36), 36)

In [67]:
print(classification_report(y_test.iloc[:, 1:].values, np.array([x[1:] for x in y_pred])))

              precision    recall  f1-score   support

           0       0.88      0.45      0.60      1131
           1       0.00      0.00      0.00        33
           2       0.79      0.62      0.70      2722
           3       0.65      0.06      0.11       540
           4       0.80      0.07      0.13       328
           5       0.86      0.03      0.07       172
           6       0.00      0.00      0.00       120
           7       0.63      0.05      0.10       232
           8       0.00      0.00      0.00         0
           9       0.93      0.23      0.37       427
          10       0.87      0.48      0.62       729
          11       0.85      0.24      0.38       596
          12       0.86      0.07      0.14        80
          13       1.00      0.04      0.07       171
          14       1.00      0.01      0.03        75
          15       0.75      0.01      0.03       206
          16       0.90      0.12      0.21       298
          17       0.47    

C:\Users\ruoru\.conda\envs\style-transfer\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ruoru\.conda\envs\style-transfer\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ruoru\.conda\envs\style-transfer\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ruo

### 6. Improve your model
Use grid search to find better parameters. 

In [66]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x00000208A82C4AF0>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier(n_jobs=-1)))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x00000208A82C4AF0>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier(n_jobs=-1)),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__subl

In [73]:
parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),
    'vect__ngram_range': ((1, 1), (1,2)),
    'vect__stop_words':(None,'english'),
    'vect__max_features': (None, 5000,10000),
    'tfidf__use_idf': (True, False)
}
cv = GridSearchCV(pipeline, param_grid=parameters)



Wall time: 80.6 ms


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [74]:
%%time
cv.fit(X_train,y_train)
print('Best Parameters:', cv.best_params_)

C:\Users\ruoru\.conda\envs\style-transfer\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(
C:\Users\ruoru\.conda\envs\style-transfer\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(
C:\Users\ruoru\.conda\envs\style-transfer\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(
C:\Users\ruoru\.conda\envs\style-transfer\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocess

C:\Users\ruoru\.conda\envs\style-transfer\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(
C:\Users\ruoru\.conda\envs\style-transfer\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(
C:\Users\ruoru\.conda\envs\style-transfer\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(
C:\Users\ruoru\.conda\envs\style-transfer\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocess

C:\Users\ruoru\.conda\envs\style-transfer\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(
C:\Users\ruoru\.conda\envs\style-transfer\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(
C:\Users\ruoru\.conda\envs\style-transfer\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(
C:\Users\ruoru\.conda\envs\style-transfer\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocess

C:\Users\ruoru\.conda\envs\style-transfer\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(
C:\Users\ruoru\.conda\envs\style-transfer\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(
C:\Users\ruoru\.conda\envs\style-transfer\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(
C:\Users\ruoru\.conda\envs\style-transfer\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocess

C:\Users\ruoru\.conda\envs\style-transfer\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(
C:\Users\ruoru\.conda\envs\style-transfer\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(
C:\Users\ruoru\.conda\envs\style-transfer\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(
C:\Users\ruoru\.conda\envs\style-transfer\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocess

C:\Users\ruoru\.conda\envs\style-transfer\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(
C:\Users\ruoru\.conda\envs\style-transfer\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(
C:\Users\ruoru\.conda\envs\style-transfer\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(
C:\Users\ruoru\.conda\envs\style-transfer\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocess

C:\Users\ruoru\.conda\envs\style-transfer\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(


Best Parameters: {'tfidf__use_idf': True, 'vect__max_df': 0.75, 'vect__max_features': 5000, 'vect__ngram_range': (1, 2), 'vect__stop_words': 'english'}
Wall time: 10h 54min 48s


In [ ]:
#Best Parameters: {'tfidf__use_idf': True, 'vect__max_df': 0.75, 'vect__max_features': 5000, 'vect__ngram_range': (1, 2), 'vect__stop_words': 'english'}
#Wall time: 10h 54min 48s

In [76]:
%%time
y_pred = cv.predict(X_test)


Wall time: 4.52 s


In [78]:
len(y_pred)

6554

In [85]:
print(classification_report(y_test.iloc[:, 1:].values, np.array([x[1:] for x in y_pred])))

              precision    recall  f1-score   support

           0       0.79      0.50      0.61      1131
           1       0.00      0.00      0.00        33
           2       0.74      0.70      0.72      2722
           3       0.63      0.16      0.25       540
           4       0.75      0.20      0.31       328
           5       0.74      0.13      0.23       172
           6       0.17      0.01      0.02       120
           7       0.55      0.11      0.19       232
           8       0.00      0.00      0.00         0
           9       0.85      0.55      0.67       427
          10       0.80      0.74      0.77       729
          11       0.81      0.50      0.62       596
          12       0.83      0.24      0.37        80
          13       0.86      0.04      0.07       171
          14       1.00      0.03      0.05        75
          15       0.58      0.17      0.26       206
          16       0.76      0.34      0.47       298
          17       0.61    

C:\Users\ruoru\.conda\envs\style-transfer\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ruoru\.conda\envs\style-transfer\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ruoru\.conda\envs\style-transfer\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ruo

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

Randomforest takes forever to run, so from <here>(https://scikit-learn.org/stable/modules/sgd.html) I decide to use another ML method called Stochastic Gradient Descent (SGD)
    
other feature could be like hashvectorizer, but it was running for too long, so I gave up and tried another two ML Algorithms, MultinomialNB and AdanboostClassifier, so far AdaboostClassifier's performace is alomost like random forest and with way less run time

In [112]:
from sklearn.naive_bayes import MultinomialNB

In [190]:
pipeline_MNB = Pipeline([
    ('vect',CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf',MultiOutputClassifier(MultinomialNB())),
])

In [191]:
%%time
pipeline_MNB.fit(X_train,y_train)

Wall time: 5.49 s


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x00000208A82C4AF0>)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(estimator=MultinomialNB()))])

In [192]:
y_pred_MNB = pipeline_MNB.predict(X_test)

In [193]:
y_pred_MNB.shape,y_test.shape,len(list(Y.columns))

((6554, 36), (6554, 36), 36)

In [194]:
print(classification_report(y_test.iloc[:, 1:].values, np.array([x[1:] for x in y_pred_MNB])))

              precision    recall  f1-score   support

           0       0.89      0.20      0.32      1131
           1       0.00      0.00      0.00        33
           2       0.75      0.59      0.66      2722
           3       0.50      0.00      0.00       540
           4       0.00      0.00      0.00       328
           5       0.00      0.00      0.00       172
           6       0.00      0.00      0.00       120
           7       0.00      0.00      0.00       232
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00       427
          10       0.70      0.01      0.02       729
          11       0.00      0.00      0.00       596
          12       0.00      0.00      0.00        80
          13       0.00      0.00      0.00       171
          14       0.00      0.00      0.00        75
          15       0.00      0.00      0.00       206
          16       0.00      0.00      0.00       298
          17       0.00    

C:\Users\ruoru\.conda\envs\style-transfer\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ruoru\.conda\envs\style-transfer\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ruoru\.conda\envs\style-transfer\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ruo

In [211]:
pipeline_Ada = Pipeline([
    ('vect',CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf',MultiOutputClassifier(AdaBoostClassifier()))
])

In [212]:
%%time
pipeline_Ada.fit(X_train,y_train)

Wall time: 1min 33s


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x00000208A82C4AF0>)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(estimator=AdaBoostClassifier()))])

In [213]:
y_pred_ada = pipeline_Ada.predict(X_test)

In [215]:
print(classification_report(y_test.iloc[:, 1:].values, np.array([x[1:] for x in y_pred_ada])))

              precision    recall  f1-score   support

           0       0.77      0.56      0.65      1131
           1       0.00      0.00      0.00        33
           2       0.76      0.60      0.67      2722
           3       0.66      0.28      0.40       540
           4       0.65      0.30      0.41       328
           5       0.65      0.21      0.32       172
           6       0.21      0.03      0.06       120
           7       0.58      0.31      0.40       232
           8       0.00      0.00      0.00         0
           9       0.77      0.58      0.66       427
          10       0.78      0.69      0.73       729
          11       0.78      0.56      0.65       596
          12       0.66      0.39      0.49        80
          13       0.68      0.25      0.37       171
          14       0.50      0.17      0.26        75
          15       0.53      0.27      0.36       206
          16       0.68      0.47      0.56       298
          17       0.52    

C:\Users\ruoru\.conda\envs\style-transfer\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ruoru\.conda\envs\style-transfer\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ruoru\.conda\envs\style-transfer\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ruo

In [218]:
pipeline_Ada.get_params()
parameters_Ada = {
    'vect__stop_words': (None,True),
    'vect__max_features': (None, 5000,10000),
    'tfidf__use_idf': (True, False)
}


In [219]:
cv_Ada = GridSearchCV(pipeline_Ada, param_grid=parameters_Ada)

In [220]:
%%time
cv_Ada.fit(X_train,y_train)
print('Best Parameters:', cv_Ada.best_params_)

C:\Users\ruoru\.conda\envs\style-transfer\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\ruoru\.conda\envs\style-transfer\lib\site-packages\sklearn\model_selection\_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\ruoru\.conda\envs\style-transfer\lib\site-packages\sklearn\pipeline.py", line 390, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "C:\Users\ruoru\.conda\envs\style-transfer\lib\site-packages\sklearn\pipeline.py", line 348, in _fit
    X, fitted_tr

Best Parameters: {'tfidf__use_idf': True, 'vect__max_features': 10000, 'vect__stop_words': None}
Wall time: 33min 31s


In [ ]:
#Best Parameters: {'tfidf__use_idf': True, 'vect__max_features': 10000, 'vect__stop_words': None}
#Wall time: 33min 31s

In [221]:
%%time
y_pred_cv_ada = cv_Ada.predict(X_test)

Wall time: 4.22 s


In [223]:
print(classification_report(y_test.iloc[:, 1:].values, np.array([x[1:] for x in y_pred_cv_ada])))

              precision    recall  f1-score   support

           0       0.77      0.55      0.64      1131
           1       0.00      0.00      0.00        33
           2       0.76      0.60      0.67      2722
           3       0.66      0.27      0.39       540
           4       0.66      0.32      0.43       328
           5       0.60      0.16      0.26       172
           6       0.23      0.04      0.07       120
           7       0.58      0.31      0.41       232
           8       0.00      0.00      0.00         0
           9       0.73      0.59      0.65       427
          10       0.80      0.68      0.73       729
          11       0.77      0.55      0.64       596
          12       0.76      0.51      0.61        80
          13       0.64      0.22      0.32       171
          14       0.73      0.21      0.33        75
          15       0.59      0.25      0.35       206
          16       0.65      0.41      0.50       298
          17       0.52    

C:\Users\ruoru\.conda\envs\style-transfer\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ruoru\.conda\envs\style-transfer\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ruoru\.conda\envs\style-transfer\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ruo

### 9. Export your model as a pickle file

In [225]:
import joblib

In [226]:
#save best parm model from random forest model.
joblib.dump(cv, 'randomF.pkl')
joblib.dump(cv.best_estimator_, 'randomF_best.pkl')

['randomF_best.pkl']

In [227]:
#save ada model with the best estimator
joblib.dump(cv_Ada, 'Ada.pkl')
joblib.dump(cv_Ada.best_estimator_, 'Ada_best.pkl')

['Ada_best.pkl']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.